In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

import censusbatchgeocoder

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
import sys
!{sys.executable} -mpip install -q censusbatchgeocoder

In [3]:
pkg = mp.jupyter.open_package()
#pkg = mp.jupyter.open_source_package()
pkg

# HIC Geocodes for San Diego CoC
`sandiegodata.org-hic_geocode-1` Last Update: 2019-07-15T22:03:07

_Geocoded addresses for housing facilities in the San Diego CoC._




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)


## References

* **[hic_source](metapack+http://library.metatab.org/hudexchange.info-hic-project-1.zip#hic)**.

In [19]:
r = pkg.reference('hic_source').dataframe()
t = r[['address1','city','state','zip']].dropna().drop_duplicates()
t.columns = 'address,city,state,zipcode'.split(',')
t.index.name = 'id'
t.to_csv('addresses.csv')

results = censusbatchgeocoder.geocode('addresses.csv', batch_size=5000)

/Users/eric/proj/code-projects/python-censusbatchgeocoder/censusbatchgeocoder/__init__.py:226: ResourceWarning: unclosed file <_io.TextIOWrapper name='addresses.csv' mode='r' encoding='UTF-8'>


In [52]:
df = pd.DataFrame(results)
df['id'] = pd.to_numeric(df.id).astype(int)
df.set_index('id', inplace=True)
df.head()

,address,city,state,zipcode,geocoded_address,is_match,is_exact,returned_address,coordinates,tiger_line,side,state_fips,county_fips,tract,block,longitude,latitude
id,,,,,,,,,,,,,,,,,
4252,165 Chief Eddie Hoffman Hwy,Bethel,AK,99559.0,"165 Chief Eddie Hoffman Hwy, Bethel, AK, 99559.0",Match,Exact,"165 CHIEF EDDIE HOFFMAN HWY, BETHEL, AK, 99559","-161.76959,60.792988",624323117,R,02,050,000200,3004,-161.76959,60.792988
4253,1801 FISHHOOK RD,WASILLA,AK,99654.0,"1801 FISHHOOK RD, WASILLA, AK, 99654.0",No_Match,None,None,None,None,None,None,None,None,None,NaN,NaN
4254,410 THORSHEIM ST,KODIAK,AK,99615.0,"410 THORSHEIM ST, KODIAK, AK, 99615.0",Match,Exact,"410 THORSHEIM ST, KODIAK, AK, 99615","-152.40607,57.793",193545252,R,02,150,000300,1014,-152.40607,57.793000
4255,450 Wittier Street,Juneau,AK,99801.0,"450 Wittier Street, Juneau, AK, 99801.0",No_Match,None,None,None,None,None,None,None,None,None,NaN,NaN
4256,723 27TH AVE,FAIRBANKS,AK,99701.0,"723 27TH AVE, FAIRBANKS, AK, 99701.0",Match,Exact,"723 27TH AVE, FAIRBANKS, AK, 99701","-147.71657,64.822464",190894149,L,02,090,000300,4034,-147.71657,64.822464


In [154]:
idx = r[['year','organization_id', 'project_id', 'coc_id','hudnum','row', 'address1']].join(df, how='left')

from shapely.geometry import Point
import geopandas as  gpd

idx['geometry'] = [Point(x,y) for x,y in zip(idx.longitude, idx.latitude) if x and y] #gpd.GeoDataFrame(idx, geometry=)

from geoid.acs import Tract, Blockgroup

def mktract(state, county, tract):
    try:
        return Tract(int(state), int(county), int(tract) )
    except Exception:
        return None
    
def mkbg(state, county, tract, block):
    try:
        return Blockgroup(int(state), int(county), int(tract), int(block) )
    except Exception:
        return None  

idx['geoid'] = idx.apply(lambda r: mktract(r.state_fips, r.county_fips, r.tract) ,axis=1)

idx = idx[['row','year','organization_id','project_id', 'coc_id', 'hudnum',  'is_match','is_exact','returned_address',
 'geoid','longitude', 'latitude', 'state_fips','county_fips', 'geometry']].copy()

idx.to_csv('../data/hic_geocode.csv', index=False)



In [118]:
gdf = gpd.GeoDataFrame(idx)

In [122]:
ca = gdf[(gdf.state_fips == '06') & (gdf.county_fips == '073')]

In [149]:
idx[['coc_id', 'hudnum']].drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 435 entries, 0 to 165420
Data columns (total 2 columns):
coc_id    435 non-null int64
hudnum    435 non-null object
dtypes: int64(1), object(1)
memory usage: 10.2+ KB
